# Homework 2: Two ways with MIMIC-III
### Jonathan Wu
### Due: 5/9/2025

#### Part I (Relational)
1. Load all the data into a duckdb database. Refer to MIMIC-III example for guidance to create your own database.
2. Answer the following analysis questions along with your queries.
For each question, provide the following four pieces of information:
<br>a. the SQL query,
<br>b. brief explanation of the query (i.e., what operations are performed by the major parts of the query),
<br>c. the first several lines of your resulting table, and
<br>d. a summary of your findings. If it benefits to use a graph, include your graph at the end of your answer, with clear labels and caption.

The analysis questions are:

1. Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.
2. Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.
3. How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

In [ ]:
!pip install duckdb==1.2.2
import duckdb
import pandas as pd

In [ ]:
conn = duckdb.connect('mimic.db', read_only=False)

conn.sql('SHOW TABLES;')

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ DRGCODES   │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [ ]:
df = pd.read_csv("PRESCRIPTIONS.csv", index_col="row_id")
df.head()

,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
row_id,,,,,,,,,,,,,,,,,,
32600,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,PNEUMOcoccal Vac Polyvalent,PNEU25I,48548.0,6494300.0,25mcg/0.5mL Vial,0.5,mL,1,VIAL,IM
32601,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl,BISA5,2947.0,536338101.0,5 mg Tab,10,mg,2,TAB,PO
32602,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Bisacodyl,Bisacodyl,Bisacodyl (Rectal),BISA10R,2944.0,574705050.0,10mg Suppository,10,mg,1,SUPP,PR
32603,42458,159647,NaN,2146-07-21 00:00:00,2146-07-22 00:00:00,MAIN,Senna,Senna,Senna,SENN187,19964.0,904516561.0,1 Tablet,1,TAB,1,TAB,PO
32604,42458,159647,NaN,2146-07-21 00:00:00,2146-07-21 00:00:00,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,3017.0,121054410.0,100mg UD Cup,100,mg,1,UDCUP,PO


In [ ]:
df.to_sql("PRESCRIPTIONS", conn, index=False)

<ipython-input-5-b6ee6ec9addd>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql("PRESCRIPTIONS", conn, index=False)


-1

In [ ]:
conn.close()

In [ ]:
conn = duckdb.connect('mimic.db')

conn.sql('SHOW TABLES;')

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ ADMISSIONS    │
│ DRGCODES      │
│ D_ICDPROCS    │
│ ICUSTAYS      │
│ PATIENTS      │
│ PRESCRIPTIONS │
│ PROCS_ICD     │
└───────────────┘

In [ ]:
conn.sql(
    """
    SELECT * FROM PRESCRIPTIONS LIMIT 2;
    """
)

┌────────────┬─────────┬────────────┬─────────────────────┬─────────────────────┬───────────┬─────────────────────────────┬─────────────────────────────┬─────────────────────────────┬───────────────────┬─────────┬─────────────┬──────────────────┬─────────────┬──────────────┬───────────────┬────────────────┬─────────┐
│ subject_id │ hadm_id │ icustay_id │      startdate      │       enddate       │ drug_type │            drug             │        drug_name_poe        │      drug_name_generic      │ formulary_drug_cd │   gsn   │     ndc     │  prod_strength   │ dose_val_rx │ dose_unit_rx │ form_val_disp │ form_unit_disp │  route  │
│   int32    │  int32  │   float    │       varchar       │       varchar       │  varchar  │           varchar           │           varchar           │           varchar           │      varchar      │  float  │    float    │     varchar      │   varchar   │   varchar    │    varchar    │    varchar     │ varchar │
├────────────┼─────────┼────────────┼──────

#### Part I Question 1
Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

In [ ]:
conn.sql(
    """
    SELECT ethnicity, drug, dose_val_rx, dose_unit_rx, count(drug) as prescription_count
    FROM ADMISSIONS
    JOIN PRESCRIPTIONS ON ADMISSIONS.subject_id = PRESCRIPTIONS.subject_id
    GROUP BY ethnicity, drug, dose_val_rx, dose_unit_rx
    ORDER BY prescription_count DESC
    LIMIT 5;
    """
)

┌────────────────────────────────┬──────────────────────────────────────┬─────────────┬──────────────┬────────────────────┐
│           ethnicity            │                 drug                 │ dose_val_rx │ dose_unit_rx │ prescription_count │
│            varchar             │               varchar                │   varchar   │   varchar    │       int64        │
├────────────────────────────────┼──────────────────────────────────────┼─────────────┼──────────────┼────────────────────┤
│ HISPANIC/LATINO - PUERTO RICAN │ 0.9% Sodium Chloride (Mini Bag Plus) │ 100         │ mL           │                465 │
│ HISPANIC/LATINO - PUERTO RICAN │ 0.9% Sodium Chloride                 │ 1000        │ mL           │                420 │
│ HISPANIC/LATINO - PUERTO RICAN │ Potassium Chloride                   │ 40          │ mEq          │                390 │
│ HISPANIC/LATINO - PUERTO RICAN │ Insulin                              │ 0           │ UNIT         │                375 │
│ HISPAN

Going to need to grab admissions for ethnicity and prescriptions

https://www.geeksforgeeks.org/how-to-select-the-first-row-of-each-group-by-in-sql/

In [ ]:
conn.sql(
    """
    SELECT ethnicity, drug, dose_val_rx, dose_unit_rx, prescription_count
    FROM (
        SELECT
            ADMISSIONS.ethnicity,
            PRESCRIPTIONS.drug,
            PRESCRIPTIONS.dose_val_rx,
            PRESCRIPTIONS.dose_unit_rx,
            COUNT(*) AS prescription_count,
            ROW_NUMBER() OVER (
                PARTITION BY ADMISSIONS.ethnicity
                ORDER BY COUNT(*) DESC
            ) AS rank
        FROM ADMISSIONS
        JOIN PRESCRIPTIONS ON ADMISSIONS.subject_id = PRESCRIPTIONS.subject_id
        GROUP BY
            ADMISSIONS.ethnicity,
            PRESCRIPTIONS.drug,
            PRESCRIPTIONS.dose_val_rx,
            PRESCRIPTIONS.dose_unit_rx
    )
    WHERE rank = 1
    ORDER BY ethnicity;
    """
)


┌──────────────────────────────────────────────────────────┬──────────────────────────────────────┬─────────────┬──────────────┬────────────────────┐
│                        ethnicity                         │                 drug                 │ dose_val_rx │ dose_unit_rx │ prescription_count │
│                         varchar                          │               varchar                │   varchar   │   varchar    │       int64        │
├──────────────────────────────────────────────────────────┼──────────────────────────────────────┼─────────────┼──────────────┼────────────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 5% Dextrose                          │ 1000        │ mL           │                 30 │
│ ASIAN                                                    │ D5W                                  │ 250         │ ml           │                 14 │
│ BLACK/AFRICAN AMERICAN                                   │ Insulin                              │ 

In [ ]:
conn.sql(
    """
    SELECT
        ADMISSIONS.ethnicity,
        PRESCRIPTIONS.drug,
        SUM(CAST(PRESCRIPTIONS.dose_val_rx AS INT)) AS total_dose,
        dose_unit_rx
    FROM ADMISSIONS
    JOIN PRESCRIPTIONS ON ADMISSIONS.subject_id = PRESCRIPTIONS.subject_id
    WHERE PRESCRIPTIONS.dose_val_rx ~ '^\\d+(\\.\\d+)?$'
    GROUP BY ADMISSIONS.ethnicity, PRESCRIPTIONS.drug, PRESCRIPTIONS.dose_unit_rx
    ORDER BY ethnicity, total_dose DESC;
    """
)


┌──────────────────────────────────────────────────────────┬─────────────────────────┬────────────┬──────────────┐
│                        ethnicity                         │          drug           │ total_dose │ dose_unit_rx │
│                         varchar                          │         varchar         │   int128   │   varchar    │
├──────────────────────────────────────────────────────────┼─────────────────────────┼────────────┼──────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 5% Dextrose             │      33800 │ mL           │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Heparin                 │      30000 │ UNIT         │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 0.9% Sodium Chloride    │      27200 │ mL           │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Lactulose Enema         │      10000 │ mL           │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ Vancomycin         

#### Part I Question 2
Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

In [ ]:
conn.sql(
    """
    SELECT * FROM PATIENTS LIMIT 2;
    """
)

┌────────┬────────────┬─────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────┐
│ row_id │ subject_id │ gender  │         dob         │         dod         │      dod_hosp       │       dod_ssn       │ expire_flag │
│ int32  │   int32    │ varchar │       varchar       │       varchar       │       varchar       │       varchar       │    int32    │
├────────┼────────────┼─────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────┤
│   9467 │      10006 │ F       │ 2094-03-05 00:00:00 │ 2165-08-12 00:00:00 │ 2165-08-12 00:00:00 │ 2165-08-12 00:00:00 │           1 │
│   9472 │      10011 │ F       │ 2090-06-05 00:00:00 │ 2126-08-28 00:00:00 │ 2126-08-28 00:00:00 │ NULL                │           1 │
└────────┴────────────┴─────────┴─────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┴─────────────┘

In [ ]:
conn.sql(
    """
    SELECT * FROM ADMISSIONS LIMIT 2;
    """
)

┌────────┬────────────┬─────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────┬───────────────────────────┬────────────────────┬───────────┬──────────┬──────────┬────────────────┬────────────────────────┬─────────────────────┬─────────────────────┬─────────────┬──────────────────────┬──────────────────────┐
│ row_id │ subject_id │ hadm_id │      admittime      │      dischtime      │      deathtime      │ admission_type │    admission_location     │ discharge_location │ insurance │ language │ religion │ marital_status │       ethnicity        │      edregtime      │      edouttime      │  diagnosis  │ hospital_expire_flag │ has_chartevents_data │
│ int32  │   int32    │  int32  │       varchar       │       varchar       │       varchar       │    varchar     │          varchar          │      varchar       │  varchar  │ varchar  │ varchar  │    varchar     │        varchar         │       varchar       │       varchar       │   varchar   │        i

In [ ]:
conn.sql(
    """
    SELECT * FROM PROCS_ICD LIMIT 2;
    """
)

┌────────┬────────────┬─────────┬─────────┬───────────┐
│ row_id │ subject_id │ hadm_id │ seq_num │ icd9_code │
│ int32  │   int32    │  int32  │  int32  │   int32   │
├────────┼────────────┼─────────┼─────────┼───────────┤
│   3994 │      10114 │  167957 │       1 │      3605 │
│   3995 │      10114 │  167957 │       2 │      3722 │
└────────┴────────────┴─────────┴─────────┴───────────┘

In [ ]:
conn.sql(
    """
    SELECT * FROM D_ICDPROCS LIMIT 2;
    """
)

┌────────┬───────────┬──────────────────────────┬───────────────────────────────────────────────────────────────────┐
│ row_id │ icd9_code │       short_title        │                            long_title                             │
│ int32  │   int32   │         varchar          │                              varchar                              │
├────────┼───────────┼──────────────────────────┼───────────────────────────────────────────────────────────────────┤
│      1 │      1423 │ Chorioret les xenon coag │ Destruction of chorioretinal lesion by xenon arc photocoagulation │
│      2 │      1424 │ Chorioret les laser coag │ Destruction of chorioretinal lesion by laser photocoagulation     │
└────────┴───────────┴──────────────────────────┴───────────────────────────────────────────────────────────────────┘

In [ ]:
conn.sql(
    """
    SELECT date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) as age
    FROM PATIENTS
    JOIN ADMISSIONS ON PATIENTS.subject_id=ADMISSIONS.subject_id
    LIMIT 10;
    """
)

┌───────┐
│  age  │
│ int64 │
├───────┤
│    70 │
│    36 │
│    87 │
│    74 │
│    49 │
│   300 │
│    82 │
│    78 │
│    88 │
│    81 │
└───────┘

In [ ]:
conn.sql(
    """
    SELECT CASE
    WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <= 19 THEN 'MINOR'
    WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 20 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <50 THEN 'ADULT'
    WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >=50 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 80 THEN 'OLD'
    ELSE 'ELDERLY'
    END AS age_group,
    long_title,
    FROM PATIENTS
    JOIN ADMISSIONS ON PATIENTS.subject_id=ADMISSIONS.subject_id
    JOIN PROCS_ICD ON ADMISSIONS.hadm_id=PROCS_ICD.hadm_id
    JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE=D_ICDPROCS.ICD9_CODE
    GROUP BY age_group
    LIMIT 5;
    """
)

┌───────────┬─────────────────────────────────────────────────────────────────────────────┬─────────────────┐
│ age_group │                                 long_title                                  │ procedure_count │
│  varchar  │                                   varchar                                   │      int64      │
├───────────┼─────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│ ADULT     │ Venous catheterization, not elsewhere classified                            │               9 │
│ ADULT     │ Enteral infusion of concentrated nutritional substances                     │               7 │
│ ADULT     │ Insertion of endotracheal tube                                              │               6 │
│ ADULT     │ Percutaneous abdominal drainage                                             │               6 │
│ ADULT     │ Continuous invasive mechanical ventilation for 96 consecutive hours or more │               6 │
└─────────

In [ ]:
conn.sql(
    """
    SELECT *
    FROM (
        SELECT
            CASE
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <= 19 THEN 'MINOR'
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 20 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 50 THEN 'ADULT'
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 50 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 80 THEN 'OLD'
                ELSE 'ELDERLY'
            END AS age_group,
            long_title,
            COUNT(*) AS procedure_count,
            ROW_NUMBER() OVER (
                PARTITION BY
                    CASE
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <= 19 THEN 'MINOR'
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 20 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 50 THEN 'ADULT'
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 50 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 80 THEN 'OLD'
                        ELSE 'ELDERLY'
                    END
                ORDER BY COUNT(*) DESC
            ) AS rank
        FROM PATIENTS
        JOIN ADMISSIONS ON PATIENTS.subject_id = ADMISSIONS.subject_id
        JOIN PROCS_ICD ON ADMISSIONS.hadm_id = PROCS_ICD.hadm_id
        JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
        GROUP BY age_group, long_title
    ) ranked
    WHERE rank <= 3
    ORDER BY age_group, procedure_count DESC;
    """
)

┌───────────┬─────────────────────────────────────────────────────────┬─────────────────┬───────┐
│ age_group │                       long_title                        │ procedure_count │ rank  │
│  varchar  │                         varchar                         │      int64      │ int64 │
├───────────┼─────────────────────────────────────────────────────────┼─────────────────┼───────┤
│ ADULT     │ Venous catheterization, not elsewhere classified        │               9 │     1 │
│ ADULT     │ Enteral infusion of concentrated nutritional substances │               7 │     2 │
│ ADULT     │ Insertion of endotracheal tube                          │               6 │     3 │
│ ELDERLY   │ Venous catheterization, not elsewhere classified        │              20 │     1 │
│ ELDERLY   │ Transfusion of packed cells                             │              13 │     2 │
│ ELDERLY   │ Insertion of endotracheal tube                          │               8 │     3 │
│ MINOR     │ Venous

#### Part I Question 3
How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

https://duckdb.org/docs/stable/sql/data_types/typecasting

https://duckdb.org/docs/stable/sql/data_types/interval

https://duckdb.org/docs/stable/sql/functions/datepart.html

https://duckdb.org/docs/stable/sql/functions/interval.html

In [ ]:
conn.sql(
    """
    SELECT * FROM PATIENTS LIMIT 5;
    """
)

┌────────┬────────────┬─────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────┐
│ row_id │ subject_id │ gender  │         dob         │         dod         │      dod_hosp       │       dod_ssn       │ expire_flag │
│ int32  │   int32    │ varchar │       varchar       │       varchar       │       varchar       │       varchar       │    int32    │
├────────┼────────────┼─────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────┤
│   9467 │      10006 │ F       │ 2094-03-05 00:00:00 │ 2165-08-12 00:00:00 │ 2165-08-12 00:00:00 │ 2165-08-12 00:00:00 │           1 │
│   9472 │      10011 │ F       │ 2090-06-05 00:00:00 │ 2126-08-28 00:00:00 │ 2126-08-28 00:00:00 │ NULL                │           1 │
│   9474 │      10013 │ F       │ 2038-09-03 00:00:00 │ 2125-10-07 00:00:00 │ 2125-10-07 00:00:00 │ 2125-10-07 00:00:00 │           1 │
│   9478 │      10017 │ F       │ 2075-09-21 00:

In [ ]:
conn.sql("""
SELECT * FROM ICUSTAYS LIMIT 5""")

┌────────┬────────────┬─────────┬────────────┬──────────┬────────────────┬───────────────┬──────────────┬─────────────┬─────────────────────┬─────────────────────┬─────────┐
│ row_id │ subject_id │ hadm_id │ icustay_id │ dbsource │ first_careunit │ last_careunit │ first_wardid │ last_wardid │       intime        │       outtime       │   los   │
│ int32  │   int32    │  int32  │   int32    │ varchar  │    varchar     │    varchar    │    int32     │    int32    │       varchar       │       varchar       │  float  │
├────────┼────────────┼─────────┼────────────┼──────────┼────────────────┼───────────────┼──────────────┼─────────────┼─────────────────────┼─────────────────────┼─────────┤
│  12742 │      10006 │  142345 │     206504 │ carevue  │ MICU           │ MICU          │           52 │          52 │ 2164-10-23 21:10:15 │ 2164-10-25 12:21:07 │  1.6325 │
│  12747 │      10011 │  105331 │     232110 │ carevue  │ MICU           │ MICU          │           15 │          15 │ 2126-08-14

In [ ]:
conn.sql("""
SELECT avg(epoch(cast(outtime as TIMESTAMP) - cast(intime as TIMESTAMP)))/3600 AS remaining_hours
FROM ICUSTAYS""")

┌────────────────────┐
│  remaining_hours   │
│       double       │
├────────────────────┤
│ 106.85906658496732 │
└────────────────────┘

In [ ]:
conn.sql("""
SELECT gender, avg(epoch(cast(outtime as TIMESTAMP) - cast(intime as TIMESTAMP)))/3600 AS remaining_hours
FROM ICUSTAYS JOIN PATIENTS on ICUSTAYS.subject_id=PATIENTS.subject_id GROUP BY gender""")

┌─────────┬────────────────────┐
│ gender  │  remaining_hours   │
│ varchar │       double       │
├─────────┼────────────────────┤
│ F       │ 132.96187389770722 │
│ M       │  84.33198630136985 │
└─────────┴────────────────────┘

In [ ]:
conn.sql("""
SELECT ethnicity, avg(epoch(cast(outtime as TIMESTAMP) - cast(intime as TIMESTAMP)))/3600 AS remaining_hours
FROM ICUSTAYS JOIN ADMISSIONS on ICUSTAYS.subject_id=ADMISSIONS.subject_id GROUP BY ethnicity""")

┌──────────────────────────────────────────────────────────┬────────────────────┐
│                        ethnicity                         │  remaining_hours   │
│                         varchar                          │       double       │
├──────────────────────────────────────────────────────────┼────────────────────┤
│ BLACK/AFRICAN AMERICAN                                   │  164.8246296296296 │
│ UNKNOWN/NOT SPECIFIED                                    │ 108.25615384615385 │
│ OTHER                                                    │ 22.225740740740743 │
│ ASIAN                                                    │           93.36125 │
│ HISPANIC OR LATINO                                       │  179.0312962962963 │
│ WHITE                                                    │  98.95345400728597 │
│ HISPANIC/LATINO - PUERTO RICAN                           │  77.83348148148148 │
│ UNABLE TO OBTAIN                                         │ 320.56888888888886 │
│ AMERICAN INDIA

## Part 2


In [ ]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

In [ ]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

### Part 1

In [ ]:
admissions = pd.read_csv("ADMISSIONS.csv")
#admissions.head()
admissions.join(pd.read_csv("PRESCRIPTIONS.csv"), on = 'subject_id',how = 'inner')

### Part 2

In [ ]:
patients = pd.read_csv("PATIENTS.csv")
#patients.head()
patients.join(pd.read_csv("ADMISSIONS.csv"), on = 'subject_id',how = 'inner')
patients.join(pd.read_csv("PROCS_ICD.csv"), on = 'hadm_id',how = 'inner')
patients.join(pd.read_csv("D_ICDPROCS.csv"), on = 'icd9_code',how = 'inner')

In [ ]:
conn.sql(
    """
    SELECT *
    FROM (
        SELECT
            CASE
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <= 19 THEN 'MINOR'
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 20 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 50 THEN 'ADULT'
                WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 50 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 80 THEN 'OLD'
                ELSE 'ELDERLY'
            END AS age_group,
            long_title,
            COUNT(*) AS procedure_count,
            ROW_NUMBER() OVER (
                PARTITION BY
                    CASE
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) <= 19 THEN 'MINOR'
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 20 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 50 THEN 'ADULT'
                        WHEN date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) >= 50 AND date_diff('year', cast(dob as DATE), cast(dischtime as DATE)) < 80 THEN 'OLD'
                        ELSE 'ELDERLY'
                    END
                ORDER BY COUNT(*) DESC
            ) AS rank
        FROM PATIENTS
        JOIN ADMISSIONS ON PATIENTS.subject_id = ADMISSIONS.subject_id
        JOIN PROCS_ICD ON ADMISSIONS.hadm_id = PROCS_ICD.hadm_id
        JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
        GROUP BY age_group, long_title
    ) ranked
    WHERE rank <= 3
    ORDER BY age_group, procedure_count DESC;
    """
)

### Part 3

In [ ]:
admissions = pd.read_csv("ADMISSIONS.csv")
#admissions.head()
admissions.join(pd.read_csv("ICUSTAYS.csv"), on = 'subject_id',how = 'inner')